In [1]:
from jira.client import JIRA
from requests_toolbelt import user_agent
from datetime import date,datetime
from deep_translator import GoogleTranslator
import pickle
import pandas as pd
from catboost import CatBoostClassifier

In [2]:
options = {'server': "https://hibernate.atlassian.net/"}
jira = JIRA(options, basic_auth=("bahriahmed94@gmail.com", "slHGJreyEZ1GCec8ZKbtEC6E"))

In [13]:
def get_projects_informations(jira,projects):
    file = open('../IA/models/encoderTypeTicket', 'rb')
    file_version = open('../IA/models/encoderTypeVersion', 'rb')
    model_cb = CatBoostClassifier(task_type='GPU', iterations=100, 
                              random_state = 2021, 
                              eval_metric="Accuracy",boost_from_average=False)
    model_participant=model_cb.load_model("../IA/models/participant")
    loaded_model = pickle.load(file)
    loaded_model_version = pickle.load(file_version)
    notifications=[]
    data=[]

    i=0
    for project in projects :
        nb_bug_month=0
        nb_bug_last_month=0
        print(i)
        notifications.append([])

        size = 100
        initial = 0
        test=False
        now=datetime.today()
        while True:
            
            start= initial*size
            issues = jira.search_issues('project={}'.format(project), start,size)
            if len(issues) == 0 or test:
                i+=1
                break
            initial += 1
            d=""
            version="Mineur"
            priorite=1
            type_ticket="Tâche"

            for issue in issues:
                
                a=int(issue.fields.created.split("-")[0])
                m=int(issue.fields.created.split("-")[1])
                j=int(issue.fields.created.split("-")[2][:2])
                d=date(a,m,j)
                if(((date(now.year,now.month,now.day)-d).days<=3 )):
                    if issue.fields.status.name!="Closed" and issue.fields.status.name!="Resolved"   :
                        if(len(issue.fields.versions)!=0):

                            v=issue.fields.versions[0].name
                        
                            
                            if(len(v.split("."))==3):
                                
                                if(v.split(".")[1]=="0" and v.split(".")[2].startswith("0")):
                                
                                    version="Majeur"
                                elif(v.split(".")[1]=="0" and not(v.split(".")[2].startswith("0"))):
                                    version="Patch"
                                elif(v.split(".")[1]!="0" and v.split(".")[2].startswith("0")):
                                    version="Mineur"

                            
                                
                            if (issue.fields.priority.name=="Trivial"):
                                priorite=1
                            elif (issue.fields.priority.name=="Minor"):
                                priorite=2
                            elif (issue.fields.priority.name=="Major"):
                                priorite=3
                            elif (issue.fields.priority.name=="Critical"):
                                priorite=4
                            elif (issue.fields.priority.name=="Blocker"):
                                priorite=5
                    
                            
                            if(issue.fields.issuetype.name=="Bug"):
                                type_ticket="Bug"
                            
                            elif(issue.fields.issuetype.name=="Task"):
                                type_ticket="Tâche"

                            elif(issue.fields.issuetype.name=="Improvement"):
                                type_ticket="Amélioration"
                                
                            elif(issue.fields.issuetype.name=="Sub-task"):
                                type_ticket="Sous-tâche"

                            elif(issue.fields.issuetype.name=="New Feature"):
                                type_ticket="Nouvelle fonctionnalité"
                           
                            elif(issue.fields.issuetype.name=="Deprecation"):
                                type_ticket="Deprecation"
                            elif(issue.fields.issuetype.name=="Epic"):
                                type_ticket="Epic"
                            elif(issue.fields.issuetype.name=="Remove Feature"):
                                type_ticket="Remove Feature"
                            elif(issue.fields.issuetype.name=="Patch"):
                                type_ticket="Patch"  

                            elif(issue.fields.issuetype.name=="Technical task"):
                                type_ticket="Technical task"
                            elif(issue.fields.issuetype.name=="Story"):
                                type_ticket="Story"  
                            
          
                        data={"Priorité":[priorite],"Type de ticket":[loaded_model.transform([type_ticket])[0]],"Jour":[j],"Mois":[m],"Annee":[a],"Nombre de Composants":[len(issue.fields.components)],"Nombre de Versions corrigées":[len(issue.fields.versions)],"Type de Version":[loaded_model_version.transform([version])[0]]}
                        df =pd.DataFrame.from_dict(data)
                        x=model_participant.predict(df)[0][0]
                        if(abs(x-len(issue.fields.customfield_10050)) ==2):
                            notifications[i].append({"project":issue.fields.project.name,"key":issue.key,"nombre de participant":str(x),"priority":"low"})
                        elif(abs(x-len(issue.fields.customfield_10050)) ==3):
                            notifications[i].append({"project":issue.fields.project.name,"key":issue.key,"nombre de participant":str(x),"priority":"medium"})
                        elif(abs(x-len(issue.fields.customfield_10050)) ==4):
                            notifications[i].append({"project":issue.fields.project.name,"key":issue.key,"nombre de participant":str(x),"priority":"heigh"})
                else : 
                    test=True
                    break 
            

            for issue in issues:
                a=int(issue.fields.created.split("-")[0])
                m=int(issue.fields.created.split("-")[1])
                j=int(issue.fields.created.split("-")[2][:2])
                d=date(a,m,j)
                if(d.year - a) * 12 + d.month - m <=1:
                    nb_bug_month+=1
                elif (d.year - a) * 12 + d.month - m ==2:
                    nb_bug_last_month+=1


    if(nb_bug_month-nb_bug_month>5):
        notifications[i].append({"project":issue.fields.project.name,"key":issue.key,"msg":"High number of bug :" +str(nb_bug_month),"priority":"low"})
    elif(nb_bug_month-nb_bug_month>10):
        notifications[i].append({"project":issue.fields.project.name,"key":issue.key,"msg":"High number of bug :" +str(nb_bug_month),"priority":"medium"})
    elif(nb_bug_month-nb_bug_month>15):
        notifications[i].append({"project":issue.fields.project.name,"key":issue.key,"msg":"High number of bug :" +str(nb_bug_month),"priority":"high"})

    file_version.close()                
    file.close()
    return notifications

In [14]:
ae=get_projects_informations(jira,["HHH"])

0


In [15]:
ae

[[{'project': 'Hibernate ORM',
   'key': 'HHH-15213',
   'nombre de participant': '3',
   'priority': 'low'},
  {'project': 'Hibernate ORM',
   'key': 'HHH-15212',
   'nombre de participant': '3',
   'priority': 'low'},
  {'project': 'Hibernate ORM',
   'key': 'HHH-15211',
   'nombre de participant': '3',
   'priority': 'low'},
  {'project': 'Hibernate ORM',
   'key': 'HHH-15210',
   'nombre de participant': '3',
   'priority': 'low'},
  {'project': 'Hibernate ORM',
   'key': 'HHH-15205',
   'nombre de participant': '3',
   'priority': 'low'}]]

In [20]:
import json

In [44]:
a=jira.search_issues('project=ANN', maxResults=1,json_result=True)["issues"][0]["fields"]["project"]['avatarUrls']["24x24"]
a

'https://hibernate.atlassian.net/rest/api/2/universal_avatar/view/type/project/avatar/10011?size=small'

In [1]:
"a" +str(5)+"b"

'a5b'